In [3]:
import pandas as pd,uuid


"""
Original DataFrames can be found:
    https://github.com/aueb-data-science/eda-llm-2023/tree/main/PaLM
    
Be mindful, that this github repo. needs access privileges!!
"""

cars_palm = pd.read_csv("../A3_csv/eda-llm-2023/PaLM/cars_generated.csv")

# Randomly select samples to match 22% of each dataframe
cars_palm = cars_palm.drop(
    columns=cars_palm.columns[0])# Drop the unamed column 
cars_palm = cars_palm.sample(frac=1, random_state=42)

elect_palm = pd.read_csv("../A3_csv/eda-llm-2023/PaLM/elections_generated.csv")
elect_palm = elect_palm.sample(frac=1, random_state=42)
elect_palm = elect_palm.drop(
    columns=elect_palm.columns[0])# Drop the unamed column 

print(f"Shape for cars dataframe is : {cars_palm.shape}")
print(f"Shape for elections dataframe is : {elect_palm.shape}")


def add_ids(df: pd.DataFrame):
    df['essay_id'] = [str(uuid.uuid4())[:8] for _ in range(len(df))]
    display(df.sample(3))
    return df

cars_palm = add_ids(cars_palm)
elect_palm = add_ids(elect_palm)

Shape for cars dataframe is : (1375, 2)
Shape for elections dataframe is : (1375, 2)


,text,generated,essay_id
1361,NaN,1,f671a4c8
16,"/Ms. Fellow Citizen,\r\n\r\nI am writing to yo...",1,6de12188
1162,Imagine being stuck in traffic again for the f...,1,0e658847


,text,generated,essay_id
11,"Dear Senator,\r\n\r\nI am writing to you today...",1,202237ae
584,"Dear Senator,\r\n\r\nI am writing to you today...",1,b38239aa
236,"Dear Senator,\r\n\r\nI am writing to you today...",1,97c9356c


In [4]:
import numpy as np


cars_palm["prompt_id"] = np.zeros(len(cars_palm), dtype=int)



elect_palm["prompt_id"] = np.ones(len(elect_palm), dtype=int)

elect_palm.sample()


,text,generated,essay_id,prompt_id
139,NaN,1,a5534cf4,1


In [5]:
cars_palm.sample()

,text,generated,essay_id,prompt_id
434,Cars have become a necessity in modern society...,1,9dea7a4e,0


In [7]:
import pandas as pd
# Import the csv we the essays

data = pd.read_csv("../A3_csv/train_essays.csv")

print(f"Data with students essays has a shape of: {data.shape}")

# Rename the 'old_column_name' to 'new_column_name'
data.rename(columns={'id': 'essay_id'}, inplace=True)

data.sample()

Data with students essays has a shape of: (1378, 4)


,essay_id,prompt_id,text,generated
177,228a014b,1,"Dear my Senator, whats the point in voting if ...",0


In [8]:
def concat_df(df1, df2):
    """
    Concatenate specified columns from two DataFrames.
    
    @param df1 (pd.DataFrame): First DataFrame
    @param df2 (pd.DataFrame): Second DataFrame
    @param columns_to_concat (list): List of column names to concatenate

    Returns: pd.DataFrame: Resulting DataFrame with concatenated values
    """
    
    columns_to_concat = ["essay_id","prompt_id", "text", "generated"]

    # Create an empty DataFrame with specified columns
    augmented_data = pd.DataFrame(columns=columns_to_concat)

    # Concatenate specified columns from the two DataFrames
    for column in columns_to_concat:
        augmented_data[column] = pd.concat([df1[column], df2[column]], axis=0, ignore_index=True)

    return augmented_data

In [9]:
aug_df = concat_df(cars_palm,data)
aug_df.shape

aug_df = concat_df(aug_df,elect_palm)
aug_df.shape

aug_df.prompt_id.value_counts()

prompt_id
0    2083
1    2045
Name: count, dtype: int64

In [12]:
aug_df.to_csv("../A3_csv/eda-llm-2023_train.csv", index=False)

In [37]:
import pandas as pd

"""
Original DataFrame can be found:
    https://www.kaggle.com/datasets/alejopaullier/argugpt
"""


machine = pd.read_csv("A3_csv/machine/machine-train.csv")

display(machine.sample())
machine.rename(columns={'id':'essay_id'}, inplace=True)
display(machine.sample())


# Select random samples to reach the 10% of the original DataFrame
machine = machine.sample(frac=0.10, random_state=42)
machine["generated"] = np.ones(len(machine), dtype=int)

print(machine.generated.value_counts())


aug_df = concat_df(aug_df, machine)
aug_df.shape


,id,prompt_id,prompt,text,model,temperature,exam_type,score,score_level
3321,gre_574,GRE-663,There are two types of laws: just and unjust. ...,I agree with the notion that individuals in a ...,text-davinci-003,0.8,gre,5,high


,essay_id,prompt_id,prompt,text,model,temperature,exam_type,score,score_level
79,weccl_441,WECCL-17,Some people think the university education is ...,There is no one answer to this question as the...,text-davinci-001,0.8,weccl,17,medium


generated
1    334
Name: count, dtype: int64


(2316, 4)

In [5]:
import pandas as pd, uuid
"""
Original DataFrame can be found:
    https://www.kaggle.com/datasets/thedrcat/daigt-proper-train-dataset
    
This dataset already has labeled 1 as generated and 0 as students
so there is not need to add another column.
For this dataset, prompt_id values are encoded starting from 3.
This choice is made to avoid using 0 and 1,
which have special meanings for this task

So, prompt_id values are incremented by 3.ii

"""




# Function to change column names and provid  a uuid    
def read_process_csv(data):
    """
    Process a DataFrame, including column renaming and adding a uuid column.
    @param: data (pd.DataFrame): Input DataFrame to be processed.
    @param: (str): Name of the file or dataset for identification.
    Returns: pd.DataFrame: Processed DataFrame.
    """
    
    # Rename columns in order to handle missmatching errors in df concatination
    data.rename(columns={'label': 'generated', 'prompt': 'prompt_id'}, inplace=True)
    display(data.sample())
    data['essay_id'] = [str(uuid.uuid4()) for _ in range(len(data))]
    # Check if 'prompt_id' column exists
    if 'prompt_id' not in data.columns:
        # Create a new 'prompt_id' column with UUIDs
        data['prompt_id'] = [str(uuid.uuid4()) for _ in range(len(data))]
    else:
        # Check if 'prompt_id' column is not empty before factorizing
        if not data['prompt_id'].empty:
            # Change the unique values of prompt_id; there were strings initially
            data['prompt_id'] = pd.factorize(data['prompt_id'])[0] + 3
    print(f"Name is: {name}")
    data.to_csv(f"../A3_csv/drcat/{name}.csv", index=False)

# Initialize a dictionary to store the file names:
file_names = ["train_drcat_01", "train_drcat_02",  "train_drcat_03",  "train_drcat_04" ]


# Iterate over names to create the datasets    
for name in file_names:
    data = pd.read_csv(f"../A3_csv/drcat/{name}.csv")
    read_process_csv(data)


,text,generated,source,fold,prompt_id,essay_id,generated,token_text,max_jaccard_sim,avg_jaccard_sim
496,One famous individuals achieved success unapol...,1,chat_gpt_moth,3,2377,571c7b29-95d1-4359-af62-541b6bdf2709,fake,"['One', 'famous', 'individuals', 'achieved', '...",0.1,0.042113


Name is: train_drcat_01


,essay_id,text,generated,source,prompt_id,fold,generated,token_text,max_jaccard_sim,avg_jaccard_sim
3154,8af49442-e903-4b23-bcea-ff0e32090774,Benefits Attending School versus Completing Sc...,1,llammistral7binstruct,12405,1,fake,"['Benefits', 'Attending', 'School', 'versus', ...",0.170616,0.041674


Name is: train_drcat_02


,essay_id,text,generated,source,prompt_id,fold,generated,token_text,max_jaccard_sim,avg_jaccard_sim
9500,ce03e0a9-34f4-4261-8c35-3004004fbe8a,Ralph Waldo Emerson famous American essayist p...,1,mistral7binstruct_v1,32193,2,fake,"['Ralph', 'Waldo', 'Emerson', 'famous', 'Ameri...",0.160784,0.054111


Name is: train_drcat_03


,essay_id,text,generated,source,prompt_id,fold,generated,token_text,max_jaccard_sim,avg_jaccard_sim
14240,81B7A7CB8829,Hey essay 'm gon na talk people rose hardship ...,1,llama2_chat,53,3,fake,"['Hey', 'essay', ""'m"", 'gon', 'na', 'talk', 'p...",0.175115,0.07049


Name is: train_drcat_04


In [2]:
import pandas as pd

# Read the dataframes
machine_test = pd.read_csv("../A3_csv/machine/machine-test.csv")
machine_dev = pd.read_csv("../A3_csv/machine/machine-dev.csv")
machine_train = pd.read_csv("../A3_csv/machine/machine-train.csv")
argu_gpt = pd.read_csv("../A3_csv/machine/argugpt.csv")
# display(machine_test, machine_dev, machine_train)
# Concatenate dataframes along rows
concatenated_df = pd.concat([machine_test[['id', 'prompt_id', 'text']],
                             machine_dev[['id', 'prompt_id', 'text']],
                             machine_train[['id', 'prompt_id', 'text']],
                             argu_gpt[['id', 'prompt_id', 'text']]])

# Display the concatenated dataframe
display(concatenated_df)

concatenated_df.to_csv("../A3_csv/machine/machine_concat.csv", index=False)

,id,prompt_id,text
0,weccl_45,WECCL-17,There are many different opinions on whether o...
1,weccl_264,WECCL-17,The university education is not only for prepa...
2,weccl_648,WECCL-17,"University education is a contentious issue, w..."
3,weccl_741,WECCL-17,University education has always been a topic o...
4,weccl_3024,WECCL-17,The college is not only a time of preparition ...
...,...,...,...
4033,gre_514,GRE-603,The notion that one must be forced to defend a...
4034,gre_1107,GRE-604,I strongly agree with the statement that menta...
4035,gre_1163,GRE-660,"In today’s world, where competition is highly ..."
4036,gre_1174,GRE-671,Education is one of the most powerful tools th...


In [40]:
aug_df = concat_df(aug_df, drcat)
aug_df.shape


(4968, 4)

In [41]:
"""
Original DataFrame can be found:
    data_generation_LM.ipynib
"""

lm_cars = pd.read_csv("A3_csv/lm_generated_cars.csv")
lm_cities = pd.read_csv("A3_csv/lm_generated_cities.csv")

lm_concat = concat_df(lm_cars, lm_cities)

In [42]:
aug_df = concat_df(aug_df,lm_cars )
aug_df.shape
display(aug_df.sample(3))

,essay_id,prompt_id,text,generated
1776,72897b46,1,"Dear Senator,\r\n\r\nI am writing to you today...",1
550,318c7ac8,1,"Dear Mr.Mrs. Senator, In light of previous res...",0
4200,C21E1457C1D4,2,Making Mona Lisa Smile\n\nMona Lisa is 83 perc...,0


In [43]:
# Reset indexing starting from 0
aug_df = aug_df.reset_index(drop=True)
display(aug_df.head(2))

,essay_id,prompt_id,text,generated
0,29dc986a,0,As more and more cities start to enforce no ca...,1
1,4b3e15c7,0,Car usage all over the world has reduced and b...,1


The following csv file will be used to train our classifiers for this particular task.

In [44]:
aug_df.to_csv("A3_csv/combined_train.csv", index=False)

In [45]:
aug_df.prompt_id.value_counts() # See the different themes of essays 

prompt_id
2           1862
0           1710
1            972
WECCL-02      30
WECCL-17      23
            ... 
1973           1
465            1
1952           1
631            1
2811           1
Name: count, Length: 676, dtype: int64

In [46]:
aug_df.generated.value_counts() # Balance between students and LLMs

generated
0    3133
1    2535
Name: count, dtype: int64

In [47]:
aug_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5668 entries, 0 to 5667
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   essay_id   5668 non-null   object
 1   prompt_id  5668 non-null   object
 2   text       5548 non-null   object
 3   generated  5668 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 177.3+ KB


### Note: 

##### The resulting dataframe's 'text' column is not preprocessed!!!